# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [2]:
spam_data = pd.read_csv('../assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)

In [3]:
spam_data.shape

(5572, 2)

In [4]:
spam_data.head()

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [6]:
def answer_one():
    spam = spam_data[spam_data['target']==1]

    return (len(spam) / len(spam_data)) * 100

In [7]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [8]:
def answer_two():
    vect = CountVectorizer().fit(X_train)
    features = vect.get_feature_names()
    
    return max(features, key=len)

In [9]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [10]:
def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    clf = MultinomialNB(alpha=0.1).fit(X_train_vect, y_train)
    predict = clf.predict(X_test_vect)
    
    return roc_auc_score(y_test, predict)

In [11]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [12]:
def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    
    feature_names = np.array(vect.get_feature_names())
    
    max_values = np.array(X_train_vect.max(0).toarray())[0]
    max_features = zip(feature_names[max_values.argsort()], np.sort(max_values))
    max_features = list(max_features)
    
    idx, values = zip(*max_features[:20])
    smallest = pd.DataFrame(values, idx)
    smallest = smallest.reset_index()
    smallest = smallest.sort_values([0, 'index'])
    
    idx, values = zip(*max_features[-20:])
    largest = pd.DataFrame(values, idx)
    largest = largest.reset_index()
    largest = largest.sort_values([0, 'index'], ascending=[False, True])
    
    return (smallest, largest)

In [13]:
answer_four()

(           index         0
 2         aaniye  0.074475
 6       athletic  0.074475
 11          chef  0.074475
 4      companion  0.074475
 12    courageous  0.074475
 3     dependable  0.074475
 10    determined  0.074475
 7   exterminator  0.074475
 1         healer  0.074475
 5       listener  0.074475
 15     organizer  0.074475
 9           pest  0.074475
 8   psychiatrist  0.074475
 14  psychologist  0.074475
 16       pudunga  0.074475
 13       stylist  0.074475
 0    sympathetic  0.074475
 17        venaam  0.074475
 18        diwali  0.091250
 19      mornings  0.091250,
         index         0
 19  146tf150p  1.000000
 2         645  1.000000
 13   anything  1.000000
 5     anytime  1.000000
 6     beerage  1.000000
 3        done  1.000000
 14         er  1.000000
 18     havent  1.000000
 17       home  1.000000
 12        lei  1.000000
 11       nite  1.000000
 8          ok  1.000000
 16       okie  1.000000
 9       thank  1.000000
 15      thanx  1.000000
 4         

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [14]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_test_vect = vect.transform(X_test)
    
    clf = MultinomialNB(alpha=0.1).fit(X_train_vect, y_train)
    predict = clf.predict_proba(X_test_vect)
    
    return roc_auc_score(y_test, predict[:,1])

In [15]:
answer_five()

0.9954968337775665

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [16]:
def answer_six():
    not_spam = spam_data[spam_data['target']==0]
    spam = spam_data[spam_data['target']==1]
    
    not_spam_len = [len(x) for x in not_spam['text']]
    spam_len = [len(x) for x in spam['text']]
    
    return (np.mean(not_spam_len), np.mean(spam_len))

In [17]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [18]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [19]:
def answer_seven():    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    
    X_train_vect = vect.transform(X_train)
    X_train_featured = add_feature(X_train_vect, [len(x) for x in X_train])
    
    X_test_vect = vect.transform(X_test)
    X_test_featured = add_feature(X_test_vect, [len(x) for x in X_test])
    
    clf = SVC(C=10000, probability=True).fit(X_train_featured, y_train)
    predict = clf.predict_proba(X_test_featured)
    
    return roc_auc_score(y_test, predict[:,1] )

In [20]:
answer_seven()

0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [21]:
def answer_eight():
    not_spam = spam_data[spam_data['target']==0]
    spam = spam_data[spam_data['target']==1]
    
    not_spam_len = [len(re.findall('\d', x)) for x in not_spam['text']]
    spam_len = [len(re.findall('\d', x)) for x in spam['text']]
    
    return (np.mean(not_spam_len), np.mean(spam_len))

In [22]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [23]:
def answer_nine():
    vect = TfidfVectorizer(min_df=5,ngram_range=(1, 3)).fit(X_train)

    X_train_vect = vect.transform(X_train)
    X_train_featured = add_feature(X_train_vect, [len(x) for x in X_train])
    X_train_featured = add_feature(X_train_featured, [len(re.findall('\d', x)) for x in X_train])
    
    X_test_vect = vect.transform(X_test)
    X_test_featured = add_feature(X_test_vect, [len(x) for x in X_test])
    X_test_featured = add_feature(X_test_featured, [len(re.findall('\d', x)) for x in X_test])
    
    clf = LogisticRegression(C=100, max_iter=1000).fit(X_train_featured, y_train)
    predict = clf.predict_proba(X_test_featured)
    
    return roc_auc_score(y_test, predict[:,1])

In [24]:
answer_nine()

0.9972964025601412

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [25]:
def answer_ten():
    not_spam = spam_data[spam_data['target']==0]
    spam = spam_data[spam_data['target']==1]
    
    not_spam_len = [len(re.findall('\W', x)) for x in not_spam['text']]
    spam_len = [len(re.findall('\W', x)) for x in spam['text']]
    
    return (np.mean(not_spam_len), np.mean(spam_len))

In [26]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [27]:
def answer_eleven():
    vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5)).fit(X_train[:2000])
    
    X_train_vect = vect.transform(X_train[:2000])
    X_train_featured = add_feature(X_train_vect, [len(x) for x in X_train[:2000]])
    X_train_featured = add_feature(X_train_featured, [len(re.findall('\d', x)) for x in X_train[:2000]])
    X_train_featured = add_feature(X_train_featured, [len(re.findall('\W', x)) for x in X_train[:2000]])
    
    X_test_vect = vect.transform(X_test)
    X_test_featured = add_feature(X_test_vect, [len(x) for x in X_test])
    X_test_featured = add_feature(X_test_featured, [len(re.findall('\d', x)) for x in X_test])
    X_test_featured = add_feature(X_test_featured, [len(re.findall('\W', x)) for x in X_test])
    
    clf = LogisticRegression(C=100, max_iter=1000).fit(X_train_featured, y_train[:2000])
    predict = clf.predict_proba(X_test_featured)
    score = roc_auc_score(y_test, predict[:,1])
    
    coefs = sorted(clf.coef_[0])
    smallest = coefs[:10]
    largest = sorted(coefs[-10:], reverse=True)
    
    return (score, smallest, largest)

In [28]:
answer_eleven()

(0.9975637913179294,
 [-0.6013535355263999,
  -0.5941947321517396,
  -0.5487789015593854,
  -0.5434396887964756,
  -0.44428400446273936,
  -0.4282935893827091,
  -0.40508263416606577,
  -0.40359810720222816,
  -0.3737632728875087,
  -0.3672546396017555],
 [1.1781686646591387,
  0.5998126011517221,
  0.594323414929016,
  0.5064815744487801,
  0.4876920304213545,
  0.4773716090032144,
  0.46509372300727186,
  0.4594081402159018,
  0.44658044545918846,
  0.4069756580114713])